In [67]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Reading data
data = pd.read_csv("output.csv")

In [4]:
# Storing data in corresponding lists
data_size = len(data)
input_position = []
input_joint = []
output_joint = []
for i in range(data_size):
    input_position.append( tuple( [float(i) for i in data['input position'][i][1:-1].split(',')] ) )
    input_joint.append( tuple( [float(i) for i in data['input joint'][i][1:-1].split(',')] ) )
    output_joint.append( tuple( [float(i) for i in data['output joint'][i][1:-1].split(',')] ) )

In [5]:
x_train = []
y_train = []
for i in range(data_size):
    x_sample = list(input_position[i])
    x_sample.extend(input_joint[i])
    x_train.append(x_sample)
    y_sample = list(output_joint[i])
    y_train.append(y_sample)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [6]:
# my model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=100, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(units=2)
])

2023-01-04 19:38:59.539061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/omaritani/catkin_ws/devel/lib:/opt/ros/noetic/lib
2023-01-04 19:38:59.579894: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-04 19:38:59.580180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (omaritani-HP-Notebook): /proc/driver/nvidia/version does not exist
2023-01-04 19:38:59.610607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate comp

In [7]:
loss = tf.keras.losses.mse
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=tf.metrics.RootMeanSquaredError())
model.fit(tf.expand_dims(x_train, -1), y_train, epochs=200, verbose=1, callbacks=[early_stop])

Epoch 1/200
121/121 [==============================] - 3s 5ms/step - loss: 0.0806 - root_mean_squared_error: 0.2839
Epoch 2/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0025 - root_mean_squared_error: 0.0498
Epoch 3/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0035 - root_mean_squared_error: 0.0591
Epoch 4/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0032 - root_mean_squared_error: 0.0565
Epoch 5/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0027 - root_mean_squared_error: 0.0523
Epoch 6/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0289 - root_mean_squared_error: 0.1701
Epoch 7/200
121/121 [==============================] - 0s 4ms/step - loss: 0.0039 - root_mean_squared_error: 0.0622
Epoch 8/200
121/121 [==============================] - 0s 3ms/step - loss: 0.0027 - root_mean_squared_error: 0.0524
Epoch 9/200
121/121 [==============================] - 0s 4ms/step - los

In [8]:
print(x_train[120])
print(y_train[120])

[-0.32  0.74 -3.13  0.86]
[-3.14  0.83]


In [9]:
model.predict( np.array([[-0.32,  0.74, -3.13,  0.86]]) )

1/1 [==============================] - 0s 460ms/step


array([[-3.1261806,  0.7782664]], dtype=float32)

In [83]:
# Checks first if the file is already saved
# If not, the model gets saved to the disk

import os.path
if os.path.isfile('arm_model.h5') is False:
    model.save('arm_model.h5')